In [1]:
import re
import os
import pandas as pd

   # Final Project
   This goal of this project is to disambiguate semantic roles in AMR that have multiple mappings in UMR (1:2,3). We have a dataest consisting of 5 parallel graphs.
   * Lindsay
   * Philippines Landslide
   * Putin
   * Edmund Pope
   * Pear Story

### Get Dataset Metrics and Split Role Counts

In [2]:
# put all files in dicts, right now just strings, should probably update to better type for the graph
umr_files = {}
amr_files = {}

umr_path = os.getcwd() + '\\UMR-data-english'
amr_path = os.getcwd() + '\\AMR-data-english'

for f in os.listdir(umr_path):
    file1 = open(umr_path + '\\' + f, 'r')
    umr_files[f] = file1.read()

for f in os.listdir(amr_path):
    file1 = open(amr_path + '\\' + f, 'r')
    amr_files[f] = file1.read()


print(umr_files)
print(amr_files)


{'lindsay-umr.txt': 'user name: jbonn\nuser id: 37\nfile language: english\nfile format: plain_text\nDoc ID in database: 1350\nexport time: 7/15/2023, 7:52:55 AM\n\n# :: snt1\t\n# Sentence:  Lindsay left in order to eat lunch .\n# sentence level graph:\n(s1l / leave-02\n    :ARG0 (s1p / person\n        :name (s1n / name :op1 "Lindsay"))\n    :Aspect Performance\n    :MODSTR FullAff\n    :purpose (s1e / eat-01\n        :ARG0 s1p\n        :ARG1 (s1l2 / lunch)\n        :Aspect Performance\n        :MODSTR FullAff))\n\n# alignment:\ns1l: 2-2\ns1p: 1-1\ns1n: 0-0\ns1e: 6-6\ns1l2: 7-7\n\n# document level annotation:\n(s1s0 / sentence\n    :temporal ((DCT :before s1l)\n            (s1l :after s1e))\n    :modal ((ROOT :MODAL AUTH)\n            (AUTH :FullAff s1l)\n            (AUTH :FullNeg s1e)))\n\n\n# :: snt2\t\n# Sentence:  If it rains , Alana won\'t water the plants .\n# sentence level graph:\n(s2w / water-01\n    :ARG0 (s2p / person\n        :name (s2n / name :op1 "Alana"))\n    :ARG1 (s2

#### Split Roles
The roles in green
<iframe src="https://drive.google.com/file/d/1iQhPIp0NwKRSu4ZUXIgIkMUF9EHHlIiQ/preview" width="640" height="480" allow="autoplay"></iframe>

In [23]:
def get_all_counts(roles_set, files_dict):
    #gets all the counts for all the items in a set of roles for all the files in files_dict
    #returns a dict with file names for keys with value of roles_ict with new counts
    roles_dict = dict.fromkeys(roles_set,0)
    files_counts = {}
    for f in files_dict.keys():
        #dict of amr roles to find
        roles_dict = roles_dict.copy()# reset the values at each iteration (dicts are mutable)
        for role in roles_dict.keys():
            roles_dict[role] = len(re.findall(r'{0}\s'.format(re.escape(role)), files_dict[f]))
        files_counts[f] = roles_dict
    return files_counts

amr_roles= {":mod",
            ":cause",
            ":part", 
            ":consist-of",
            ":source",
            ":destination",
            ":condition",
            ":concession"}

umr_roles = {":mod", #mod with a space to avoid modal
                  ":other-role",
                  ":cause",
                  ":reason",
                  ":part",
                  ":group",
                  ":material",
                  ":source",
                  ":start",
                  ":goal",
                  ":recipient",
                  ":condition",
                  ":concessive-condition",
                  ":concession"}

amr_files_counts = get_all_counts(amr_roles, amr_files)
umr_files_counts = get_all_counts(umr_roles, umr_files)
print(umr_files_counts)

{'lindsay-umr.txt': {':other-role': 0, ':reason': 0, ':goal': 0, ':source': 0, ':mod': 0, ':start': 0, ':condition': 1, ':part': 0, ':concession': 0, ':material': 0, ':concessive-condition': 0, ':group': 0, ':recipient': 0, ':cause': 0}, 'Lorelei_lorpt-024_Philippines_Landslide_2023-release.txt': {':other-role': 0, ':reason': 3, ':goal': 1, ':source': 1, ':mod': 26, ':start': 0, ':condition': 1, ':part': 2, ':concession': 2, ':material': 3, ':concessive-condition': 0, ':group': 0, ':recipient': 0, ':cause': 1}, 'lorelei_lorpt-151_putin_2023-release.txt': {':other-role': 0, ':reason': 0, ':goal': 0, ':source': 1, ':mod': 15, ':start': 0, ':condition': 0, ':part': 0, ':concession': 0, ':material': 0, ':concessive-condition': 0, ':group': 0, ':recipient': 0, ':cause': 0}, 'lorelei_lorpt-152_edmundpope_2023-release.txt': {':other-role': 1, ':reason': 1, ':goal': 1, ':source': 0, ':mod': 10, ':start': 0, ':condition': 0, ':part': 1, ':concession': 0, ':material': 0, ':concessive-condition':

In [39]:

amr_counts_df = pd.DataFrame.from_dict(amr_files_counts, orient='index')
amr_counts_df.loc['Total'] = amr_counts_df.sum()
amr_counts_df.loc[:,'Row_Total'] = amr_counts_df.sum(axis=1)
display(amr_counts_df)

,:source,:mod,:consist-of,:condition,:concession,:part,:cause,:destination,Row_Total
edmund_pope-AMR.txt,0,9,0,0,0,0,0,1,10
lindsay-AMR.txt,0,0,0,1,0,0,0,0,1
lorpt-024_Phillipines_landslide_AMR.txt,2,36,0,1,2,0,0,0,41
Pear_Story_2023-release.txt,5,18,0,0,0,2,1,0,26
putin_ENG_0152_2000_1208-AMR.txt,1,35,0,1,0,0,0,0,37
Total,8,98,0,3,2,2,1,1,115


In [38]:
umr_counts_df = pd.DataFrame.from_dict(umr_files_counts, orient='index')
umr_counts_df.loc['Total'] = umr_counts_df.sum()
umr_counts_df.loc[:,'Row_Total'] = umr_counts_df.sum(axis=1)
display(umr_counts_df)

,:other-role,:reason,:goal,:source,:mod,:start,:condition,:part,:concession,:material,:concessive-condition,:group,:recipient,:cause,Row_Total
lindsay-umr.txt,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
Lorelei_lorpt-024_Philippines_Landslide_2023-release.txt,0,3,1,1,26,0,1,2,2,3,0,0,0,1,40
lorelei_lorpt-151_putin_2023-release.txt,0,0,0,1,15,0,0,0,0,0,0,0,0,0,16
lorelei_lorpt-152_edmundpope_2023-release.txt,1,1,1,0,10,0,0,1,0,0,0,0,0,0,14
pear-AMR__of__english-umr-0004.txt,0,0,0,4,29,0,0,1,0,0,0,0,0,0,34
Total,1,4,2,6,80,0,2,4,2,3,0,0,0,1,105


In [33]:
#crete role relationship dict

amr2umr_splits = dict.fromkeys(amr_roles,0)
amr2umr_splits[":mod"] = [":mod",":other-role"]
amr2umr_splits[":cause"] = [":cause",":reason"]
amr2umr_splits[":part"] = [":part"]
amr2umr_splits[":cause"] = [":cause",":reason"]
amr2umr_splits[":source"] = [":material",":source",":start"]
amr2umr_splits[":consist-of"] = [":part",":group",":material"]
amr2umr_splits[":destination"] = [":goal",":recipient"]
amr2umr_splits[":condition"] = [":condition",":concessive-condition"]
amr2umr_splits[":concession"] = [":concession",":concessive-condition"]


In [34]:
#print out relationship graph
for i in amr2umr_splits:
    j_list = []
    for j in amr2umr_splits[i]:
        j_list.append(j +" " + str(umr_counts_df.loc["Total"][j]))

    print("AMR: ", i, amr_counts_df.loc['Total'][i],"; UMR: ",j_list)

AMR:  :source 8 ; UMR:  [':material 3', ':source 6', ':start 0']
AMR:  :mod 98 ; UMR:  [':mod 80', ':other-role 1']
AMR:  :consist-of 0 ; UMR:  [':part 4', ':group 0', ':material 3']
AMR:  :condition 3 ; UMR:  [':condition 2', ':concessive-condition 0']
AMR:  :concession 2 ; UMR:  [':concession 2', ':concessive-condition 0']
AMR:  :part 2 ; UMR:  [':part 4']
AMR:  :cause 1 ; UMR:  [':cause 1', ':reason 4']
AMR:  :destination 1 ; UMR:  [':goal 2', ':recipient 0']
